In [2]:
import pandas as pd
import numpy as np

#Configuracoes
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

Problema de aprendizado não supervesionado: clustering
- market tem todas as empresas
- portfolio 1, 2, 3 são os clientes de cada empresa.

__Descobrir:__
- Se clientes da empresa 1 estão nas empresas 2 e 3 e vice versa
- Quantidade de empresas que não estão em nenhuma das empresas
- Quantidade de campos nulos em cada um dos porfolios

__Testar:__
- SVM na base de market: testar para entender se são classes separáveis
    - Colocar label de qual empresa aquela empresa é cliente
    - Colocar label de não explorado para as que nao tem alocação
    - Aplicar PCA com SVM 
    
__Solução final:__
- Receber uma lista com clientes atuais
- Tratar os dados recebidos via pipeline
- Encontrar cluster pros dados recebidos
- Classificar nos cluster a base nova

In [3]:
market = pd.read_csv('data/estaticos_market.csv')
pf1 = pd.read_csv('data/estaticos_portfolio1.csv')
pf2 = pd.read_csv('data/estaticos_portfolio2.csv')
pf3 = pd.read_csv('data/estaticos_portfolio3.csv')

FileNotFoundError: [Errno 2] File data/estaticos_market.csv does not exist: 'data/estaticos_market.csv'

In [10]:
pf1.head()
pf1.shape

(555, 182)

In [11]:
id_1 = pf1['id'].value_counts()
id_2 = pf2['id'].value_counts()
id_3 = pf3['id'].value_counts()

In [12]:
if(id_1 == id_2):
    print("igual")

ValueError: Can only compare identically-labeled Series objects